In [10]:
import requests 
import json
"""
    bigkinds 사회, 문화 뉴스 데이터 읽어오기
    keyword : string
    startDate : yyyy-mm-dd 예)2025-07-01
    endDate : yyyy-mm-dd 예)2025-07-15
"""
def getSearchNews(keyword: str, startDate, endDate, page, wanted_code : str = "001"):
    URL = "https://www.bigkinds.or.kr/api/news/search.do"
    CODES = {  ##001: 정치, #002:경제 # 003:사회, ##004:문화
        "001":["001005000","001001000","001004000","001003000","001007000","001002000","001006000"],
        "002":["002000000","002004000","002010000","002008000","002014000","002011000","002009000","002005000","002001000","002012000","002006000","002002000","002007000","002003000","002013000"],
        "003":["003000000","003007000","003003000","003006000","003002000","003009000","003001000","003005000","003010000","003008000","003004000","004000000","004002000","004011000","004006000","004003000","004007000","004010000","004004000","004008000","004001000","004009000","004005000"],
        "004":["004000000","004002000","004011000","004006000","004003000","004007000","004010000","004004000","004008000","004001000","004009000","004005000"]
    }
    params = {
        "indexName": "news",
        "searchKey": keyword,
        "searchKeys": [{}],
        "byLine": "",
        "searchFilterType": "1",
        "searchScopeType": "1",
        "searchSortType": "date",
        "sortMethod": "date",
        "mainTodayPersonYn": "",
        "startDate": startDate,
        "endDate": endDate,
        "newsIds": [],
        "categoryCodes": CODES[wanted_code],
        "providerCodes": [],
        "incidentCodes": [],
        "networkNodeType": "",
        "topicOrigin": "",
        "dateCodes": [],
        "editorialIs": False,
        "startNo": page,
        "resultNumber": 10,
        "isTmUsable": False,
        "isNotTmUsable": False,
    }
    header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0",
        "Origin": "https://www.bigkinds.or.kr",
        "Referer": "https://www.bigkinds.or.kr/v2/news/search.do",
    }
    response = requests.post(URL, data=json.dumps(params), headers=header)
    return response.json()

In [ ]:
result = getSearchNews("", "2025-07-20", "2025-07-27", 1, "001")  # Test
result

{'getCategoryCodeList': [{'CategoryCode': '001000000',
   'CategoryCount': '17878',
   'CategoryName': '정치'},
  {'CategoryCode': '002000000', 'CategoryCount': '1495', 'CategoryName': '경제'},
  {'CategoryCode': '003000000', 'CategoryCount': '2759', 'CategoryName': '사회'},
  {'CategoryCode': '004000000', 'CategoryCount': '217', 'CategoryName': '문화'},
  {'CategoryCode': '005000000', 'CategoryCount': '1947', 'CategoryName': '국제'},
  {'CategoryCode': '006000000', 'CategoryCount': '1914', 'CategoryName': '지역'},
  {'CategoryCode': '007000000', 'CategoryCount': '32', 'CategoryName': '스포츠'},
  {'CategoryCode': '008000000',
   'CategoryCount': '122',
   'CategoryName': 'IT_과학'}],
 'documentCount': 10,
 'totalCntNotAnalysis': 1518,
 'errorMessage': None,
 'errorCode': None,
 'totalCntAnalysis': 16360,
 'totalCount': 17878,
 'getDateCodeList': [{'date': '2025', 'dateCount': '17878'}],
 'getIncidentCodeList': [{'IncidentName': '범죄',
   'IncidentCount': '1027',
   'IncidentCode': '1'},
  {'IncidentNam

# 장르별 중복 이슈 추출 (10개 이상)
- 최근 1주일간 경제, 사회, 문화 등 주요 장르별 뉴스 데이터를 수집하고, 텍스트 유사도 기반으로 중복 이슈(주제/키워드)를 추출
- 텍스트 유사도 기반 (비지도 학습) 을 통해 클러스터링을 진행
- 로직 순서 : 데이터 수집 → 전처리 → 벡터화 → 클러스터링 → 대표 이슈 추출

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import Counter
from datetime import datetime, timedelta
import re
import time
import warnings
warnings.filterwarnings('ignore')

# 분야별 코드 매핑 (정치, 경제, 사회, 문화)
GENRE_CODES = {
    '정치': '001',
    '경제': '002',
    '사회': '003',
    '문화': '004',
}

def get_all_news_for_genre(genre, start_date, end_date, wanted_code, max_per_page = 100, max_total = 1000):
    """1주일 내 모든 뉴스 데이터 수집 (최대 max_total)"""
    all_news = []
    page = 1
    while len(all_news) < max_total:
        try:
            result = getSearchNews("", start_date, end_date, page, wanted_code)
            news_list = result.get('resultList', [])
            if not news_list:
                break
            for news in news_list:
                title = news.get('TITLE', '')
                content = news.get('CONTENT', '')
                all_news.append(title + ' ' + content)
                if len(all_news) >= max_total:
                    break
            page += 1
            time.sleep(0.2)  # API 과다 호출 방지
        except Exception as e:
            print(f"Error fetching {genre} page {page}: {e}")
            break
    return all_news

def clean_text(text):
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def cluster_and_extract_duplicates(news_list, min_group_size = 10):
    docs = [clean_text(news) for news in news_list if news and clean_text(news)]
    if not docs:
        print('뉴스 데이터가 부족하거나, 전처리 후 남는 문서가 없습니다.')
        return [], 0
    # 클러스터 개수는 전체 뉴스 수의 1/10로 설정(최소 2, 최대 100)
    n_clusters = min(max(len(docs)//10, 2), 100)
    try:
        vectorizer = TfidfVectorizer(max_features = 1000, stop_words = None)
        X = vectorizer.fit_transform(docs)
        if X.shape[0] < n_clusters:
            n_clusters = X.shape[0]
        if n_clusters == 0:
            print('유효한 뉴스 문서가 없습니다.')
            return [], 0
        kmeans = KMeans(n_clusters=n_clusters, random_state = 42, n_init = 10)
        labels = kmeans.fit_predict(X)
        issues = []
        total_duplicates = 0
        for i in range(n_clusters):
            idxs = [idx for idx, label in enumerate(labels) if label == i]
            if len(idxs) < min_group_size: continue
            cluster_docs = [docs[idx] for idx in idxs]
            words = ' '.join(cluster_docs).split()
            common = Counter(words).most_common(3)
            keywords = ', '.join([w for w, _ in common])
            rep_title = cluster_docs[0][:60] + ('...' if len(cluster_docs[0]) > 60 else '')
            issues.append({'keywords': keywords, 'sample': rep_title, 'count': len(cluster_docs)})
            total_duplicates += len(cluster_docs)
        return issues, total_duplicates
    except ValueError as e:
        print(f'오류 발생: {e}')
        return [], 0

# 최근 1주일 날짜 계산
today = datetime.now()
one_week_ago = today - timedelta(days=7)
start_date = one_week_ago.strftime('%Y-%m-%d')
end_date = today.strftime('%Y-%m-%d')

genres = ['정치', '경제', '사회', '문화']
for genre in genres:
    print(f'[{genre}] 분야 뉴스 수집 중...')
    wanted_code = GENRE_CODES[genre]
    news_list = get_all_news_for_genre(genre, start_date, end_date, wanted_code, max_per_page = 100, max_total = 1000)
    print(f'뉴스 {len(news_list)}건 수집 완료')
    issues, total_duplicates = cluster_and_extract_duplicates(news_list, min_group_size = 10)
    print(f'총 중복 기사 수: {total_duplicates}')
    if not issues:
        print('중복 이슈 추출 결과가 없습니다.')
    else:
        for idx, issue in enumerate(issues, 1):
            print(f"{idx:02d}. 키워드: {issue['keywords']} (기사수: {issue['count']}) 예시: {issue['sample']}")

[정치] 분야 뉴스 수집 중...
뉴스 1000건 수집 완료
뉴스 1000건 수집 완료
총 중복 기사 수: 636
01. 키워드: 협상, 관세, 미국 (기사수: 12) 예시: 7 29 화 데일리안 출근길 뉴스 관세폭풍 목전 대통령 국익 중심 실용외교 분수령 등 관세폭풍 목전 대통령 ...
02. 키워드: 전, 김건희, 윤석열 (기사수: 11) 예시: 오늘 김건희특검 소환조사 나오나 건강악화로 출석불응 무게 김건희 여사 관련 각종 의혹을 수사하는 민중기 특별...
03. 키워드: 박찬대, 정청래, 더불어민주당 (기사수: 20) 예시: 전대 D 4 정청래 판사 평가 개편 vs 박찬대 법 왜곡 처벌 이재명 대통령의 후임 당대표를 뽑는 더불어민주...
04. 키워드: 노란봉투법, 국회, 28일 (기사수: 26) 예시: 환노위 노란봉투법 전격 처리 재계 깊은 유감 김주영 국회 환경노동위원회 소위원장이 28일 서울 여의도 국회에...
05. 키워드: 이재명, 대통령의, 28일 (기사수: 13) 예시: 이재명 대통령에 웃다가 울다가 이자놀이 한마디에 우수수 떨어진 4대금융주 보험사 증권사까지 폭락 금융업계 긴...
06. 키워드: 장관, 최휘영, 김윤덕 (기사수: 21) 예시: 오늘 문체부 국토부 장관 후보자 인사청문회 국회에서는 오늘 두 개 부처 장관 후보자에 대한 인사청문회가 열립...
07. 키워드: 신천지, 전, 국민의힘 (기사수: 16) 예시: 홍준표가 쏘아올린 신천지 연루설 민주 정당 아냐 홍준표 전 대구시장이 폭로한 신천지 국민의힘 대선 경선 개입...
08. 키워드: 정동영, 김여정, 조정 (기사수: 11) 예시: 김여정 한마디에 한미훈련 조정 꺼낸 정동영 변경사항 없어 아마도 한미 연합군사훈련이 남북관계의 가늠자가 되지...
09. 키워드: 올해, 교환권, KBS (기사수: 10) 예시: 류호정의 톱밥 먹는 중입니다 2 최저임금을 받지만 그럭저럭 괜찮다 내 전직 은 맨땅에 헤딩 이라 부를 만하다...
10. 키워드: 대통령, 전, 윤석열 (기사수: 19) 